In [13]:
import pandas as pd
import numpy as np

In [14]:
bitcoin = pd.read_csv("../data/merged_datasets.csv")

In [15]:
# Dump some columns
bitcoin.drop(inplace=True, columns=["Unnamed: 0", "Adj Close", "gold_close", "gold_label", "sp500_close", "sp500_label", \
                                    "isPartial", "compound", "neg", "neu", "pos"])

In [16]:
# Create a daily change percent
bitcoin['Daily_Change_Perc'] = bitcoin["Daily_Change"] / bitcoin["Open"]

In [17]:
delta = bitcoin['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
bitcoin['RSI'] = 100 - (100/(1 + rs))

In [18]:
bitcoin.tail(20)

,Date,Open,High,Low,Close,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,...,compound_weighted_daily_diff,compound_weighted_weekly_diff,pos_weighted_avg7,pos_weighted_daily_diff,pos_weighted_weekly_diff,neg_weighted_avg7,neg_weighted_daily_diff,neg_weighted_weekly_diff,Daily_Change_Perc,RSI
1558,2021-01-13,33915.121094,37599.960938,32584.667969,37316.359375,69364315979,3393.398437,1.0,3822.037120,-0.027116,...,-0.587706,-0.288353,0.151874,-0.146119,-0.077159,0.019089,0.027967,0.015884,0.100056,70.243547
1559,2021-01-14,37325.109375,39966.406250,36868.562500,39187.328125,63615990033,1870.968750,1.0,3849.373582,0.101800,...,0.096166,-0.126499,0.122459,0.029139,-0.018606,0.021381,-0.008894,0.004698,0.050126,72.573841
1560,2021-01-15,39156.707031,39577.710938,34659.589844,36825.367188,67760757881,-2361.960937,0.0,3638.504996,0.085559,...,0.189934,0.100250,0.119096,0.057484,0.042242,0.023874,-0.003977,-0.001772,-0.060321,65.590526
1561,2021-01-19,36642.234375,37755.890625,36069.804688,36069.804688,57244195486,-560.269531,0.0,2706.821097,-0.002995,...,0.315776,0.408702,0.129545,0.112997,0.109390,0.026586,-0.009631,-0.021874,-0.015290,63.486058
1562,2021-01-20,36050.113281,36378.328125,33570.476563,35547.750000,66834573161,-522.054688,0.0,2461.812280,-0.006804,...,-0.418428,-0.037368,0.141241,-0.082349,0.015345,0.026208,0.027618,0.006122,-0.014481,62.005713
1563,2021-01-21,35549.398438,35552.679688,30250.750000,30825.699219,75643067688,-4722.050781,0.0,1865.111152,-0.158459,...,-0.021220,-0.069460,0.143203,-0.039005,-0.025621,0.031192,0.028633,0.029771,-0.132831,50.528843
1564,2021-01-22,30817.625000,33811.851563,28953.373047,33005.761719,77207272511,2180.062500,1.0,1550.263560,-0.084948,...,-0.112823,-0.149904,0.132833,-0.028832,-0.044083,0.031063,-0.039761,-0.009861,0.070741,54.697858
1565,2021-01-25,32285.798828,34802.742188,32087.787109,32366.392578,59897054838,77.013672,1.0,739.406828,-0.019371,...,0.132727,0.016631,0.139878,0.027227,-0.016542,0.028788,-0.014432,-0.013010,0.002385,53.279744
1566,2021-01-26,32358.613281,32794.550781,31030.265625,32569.849609,60255421470,203.457031,1.0,582.815724,0.015661,...,-0.053688,0.021782,0.126744,0.023663,0.020255,0.033246,0.020138,0.002670,0.006288,53.691188
1567,2021-01-27,32564.029297,32564.029297,29367.138672,30432.546875,62576762015,-2137.302734,0.0,282.991527,-0.057506,...,-0.154901,-0.111927,0.116266,-0.063759,-0.033026,0.033437,-0.002251,0.000228,-0.065634,48.826669


In [19]:
# Weekly change (4 day trading week apparently)
bitcoin['Weekly_Change'] = bitcoin['Close'] - bitcoin['Open'].shift(4)
bitcoin['Weekly_Change_Perc'] = bitcoin['Weekly_Change'] / bitcoin['Open'].shift(4)

In [20]:
# Re-add gold
gold = pd.read_csv("../data/gold_modified.csv")
gold['Daily_Change'] = gold["Close/Last"] - gold["Open"]
gold['Daily_Change_Perc'] = gold["Daily_Change"] / gold["Open"]
gold["Increased"] = np.where(gold["Daily_Change"] > 0, 1, 0)
# Get date in right format for merge
gold["Date"] = pd.to_datetime(gold["Date"]).dt.strftime('%Y-%m-%d')
gold_needed_columns = gold[["Date", "Close/Last", "Open", "Daily_Change", "Daily_Change_Perc", "Increased"]]

In [21]:
# Merge Gold Columns Back in
bitcoin = bitcoin.merge(gold_needed_columns, on="Date", suffixes=("", "_Gold"))

In [22]:
# Re-add sp500
sp500 = pd.read_csv("../data/sp_500.csv")
sp500['Daily_Change'] = sp500["Close/Last"] - sp500["Open"]
sp500['Daily_Change_Perc'] = sp500["Daily_Change"] / sp500["Open"]
sp500["Increased"] = np.where(sp500["Daily_Change"] > 0, 1, 0)
# Get date in right format for merge
sp500["Date"] = pd.to_datetime(sp500["Date"]).dt.strftime('%Y-%m-%d')
sp_needed_columns = sp500[["Date", "Close/Last", "Open", "Daily_Change", "Daily_Change_Perc", "Increased"]]

In [23]:
bitcoin = bitcoin.merge(sp_needed_columns, on="Date", suffixes=("", "_SP500"))

In [24]:
bitcoin.tail(30)

,Date,Open,High,Low,Close,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,...,Close/Last,Open_Gold,Daily_Change_Gold,Daily_Change_Perc_Gold,Increased,Close/Last_SP500,Open_SP500,Daily_Change_SP500,Daily_Change_Perc_SP500,Increased_SP500
1548,2020-12-29,27081.810547,27370.720703,25987.298828,27362.437500,45265946774,277.628906,1.0,2137.114420,0.035004,...,1882.9,1878.0,4.9,0.002609,1,3727.04,3750.01,-22.97,-0.006125,0
1549,2020-12-30,27360.089844,28937.740234,27360.089844,28840.953125,51287442704,1478.515625,1.0,2307.615495,0.097771,...,1893.4,1881.8,11.6,0.006164,1,3732.04,3736.19,-4.15,-0.001111,0
1550,2020-12-31,28841.574219,29244.876953,28201.992188,29001.720703,46754964848,160.767578,1.0,2427.726110,0.070774,...,1895.1,1899.0,-3.9,-0.002054,0,3756.07,3733.27,22.80,0.006107,1
1551,2021-01-04,32810.949219,33440.218750,28722.755859,31971.914063,81163475344,-810.109375,0.0,3099.426762,0.088437,...,1946.6,1908.2,38.4,0.020124,1,3700.65,3764.61,-63.96,-0.016990,0
1552,2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,67547324782,2020.515625,1.0,3288.260018,0.058055,...,1954.4,1946.0,8.4,0.004317,1,3726.86,3698.02,28.84,0.007799,1
1553,2021-01-06,34013.613281,36879.699219,33514.035156,36824.363281,75289433811,2831.933593,1.0,3624.642484,0.123310,...,1908.6,1952.8,-44.2,-0.022634,0,3748.14,3712.20,35.94,0.009682,1
1554,2021-01-07,36833.875000,40180.367188,36491.191406,39371.042969,84762141031,2546.679688,1.0,4050.037469,0.231426,...,1913.6,1921.5,-7.9,-0.004111,0,3803.79,3764.71,39.08,0.010381,1
1555,2021-01-08,39381.765625,41946.738281,36838.636719,40797.609375,88107519480,1426.566406,1.0,4450.970033,0.200197,...,1835.4,1915.2,-79.8,-0.041667,0,3824.68,3815.05,9.63,0.002524,1
1556,2021-01-11,38346.531250,38346.531250,30549.599609,35566.656250,123320567399,-2789.785156,0.0,4338.167231,-0.128217,...,1850.8,1849.4,1.4,0.000757,1,3799.61,3803.14,-3.53,-0.000928,0
1557,2021-01-12,35516.359375,36568.527344,32697.976563,33922.960938,74773277909,-1643.695312,0.0,3922.051224,-0.157289,...,1844.2,1845.4,-1.2,-0.000650,0,3801.19,3801.62,-0.43,-0.000113,0


In [25]:
bitcoin = bitcoin.rename(columns={'Close/Last': 'Close/Last_Gold', 'Increased': "Increased_Gold"})

In [26]:
# set y label
bitcoin["label"] = bitcoin["Daily_Change_Ind"].shift(-1)
bitcoin.head()

,Date,Open,High,Low,Close,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,...,Open_Gold,Daily_Change_Gold,Daily_Change_Perc_Gold,Increased_Gold,Close/Last_SP500,Open_SP500,Daily_Change_SP500,Daily_Change_Perc_SP500,Increased_SP500,label
0,2014-10-13,377.920990,397.226013,368.897003,390.414001,35221400,11.864990,1.0,-15.701840,0.079798,...,1224.8,5.2,0.004246,1,1874.74,1905.65,-30.91,-0.016220,0,1.0
1,2014-10-14,391.691986,411.697998,391.324005,400.869995,38491500,10.455994,1.0,-11.895282,0.106462,...,1237.3,-3.0,-0.002425,0,1877.70,1877.11,0.59,0.000314,1,0.0
2,2014-10-15,400.954987,402.226990,388.765991,394.773010,25267100,-6.096985,0.0,-9.263747,0.042858,...,1233.3,11.5,0.009325,1,1862.49,1874.18,-11.69,-0.006237,0,1.0
3,2014-10-17,382.756012,385.477997,375.389008,383.757996,13600700,1.201996,1.0,-6.948665,-0.042687,...,1239.5,-0.5,-0.000403,0,1886.76,1864.91,21.85,0.011716,1,0.0
4,2014-10-20,389.230988,390.084015,378.252014,382.845001,16419000,-6.700989,0.0,-3.828066,-0.002379,...,1239.1,5.6,0.004519,1,1904.01,1885.62,18.39,0.009753,1,1.0


In [27]:
for (columnName, columnData) in bitcoin.iteritems():
    print('Column Name : ', columnName)
    print('Any NAs : ', columnData.isnull().any())
    try:
        print('Any INFs:', np.isinf(columnData).any())
    except:
        print("Couldn't check")

Column Name :  Date
Any NAs :  False
Couldn't check
Column Name :  Open
Any NAs :  False
Any INFs: False
Column Name :  High
Any NAs :  False
Any INFs: False
Column Name :  Low
Any NAs :  False
Any INFs: False
Column Name :  Close
Any NAs :  False
Any INFs: False
Column Name :  Volume
Any NAs :  False
Any INFs: False
Column Name :  Daily_Change
Any NAs :  False
Any INFs: False
Column Name :  Daily_Change_Ind
Any NAs :  False
Any INFs: False
Column Name :  MACD
Any NAs :  False
Any INFs: False
Column Name :  PROC_3
Any NAs :  False
Any INFs: False
Column Name :  PROC_5
Any NAs :  False
Any INFs: False
Column Name :  PROC_10
Any NAs :  False
Any INFs: False
Column Name :  wpr
Any NAs :  False
Any INFs: False
Column Name :  sto_os
Any NAs :  False
Any INFs: False
Column Name :  goog_trend_score
Any NAs :  False
Any INFs: False
Column Name :  count
Any NAs :  False
Any INFs: False
Column Name :  compound_times_retweets
Any NAs :  False
Any INFs: False
Column Name :  likes_count
Any NAs :  

In [29]:
print(len(bitcoin))
bitcoin = bitcoin.dropna()
print(len(bitcoin))

1578
1568


In [30]:
np.isinf(bitcoin['Daily_Change_Perc_SP500']).sum()

10

In [31]:
#Dump the inf
bitcoin = bitcoin[np.isinf(bitcoin['Daily_Change_Perc_SP500']) == False]

In [32]:
# Train - Test Split
from sklearn.model_selection import train_test_split
random_state = 12345

train_set, test_set = train_test_split(bitcoin, test_size=0.2, random_state=random_state)

In [33]:
len(train_set)

1246

In [34]:
len(test_set)

312

In [35]:
train_set.to_csv("../models/bitcoin_train.csv")
test_set.to_csv("../models/bitcoin_test.csv")